In [1]:
import os, sys
import autograd.numpy as np
from autograd import value_and_grad
from scipy.optimize import minimize
from util_original import get_median_inter_mnist, Kernel, load_data, ROOT_PATH, jitchol, _sqdist, \
    remove_outliers, nystrom_decomp, chol_inv
from joblib import Parallel, delayed
import time
import autograd.scipy.linalg as splg
import matplotlib.pyplot as plt

Nfeval = 1
seed = 527
np.random.seed(seed)
JITTER = 1e-7
nystr_M = 300
EYE_nystr = np.eye(nystr_M)
opt_params = None
prev_norm = None
opt_test_err = None

class autograd_data():
    def __init__(self, x, y, z):
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.z = np.asarray(z)

In [2]:
sname = 'sin'
datasize = 200
train, dev, test = load_data(ROOT_PATH + '/data/zoo/{}_{}.npz'.format(sname, datasize))
train_ad = autograd_data(train.x, train.y, train.z)
dev_ad = autograd_data(dev.x, dev.y, dev.z)

X = np.vstack((train.x, dev.x))
Y = np.vstack((train.y, dev.y))
Z = np.vstack((train.z, dev.z))
test_X = test.x
test_G = test.g

ak0 = get_median_inter_mnist(Z[:, [0]])
ak1 = get_median_inter_mnist(Z[:, [1]])
al0 = get_median_inter_mnist(X)

In [4]:
def MMR(alpha, lambdaH):
    # L = np.exp(-L0 / al / al / 2) + 1e-6 * EYEN

    alpha = alpha[:, None]
    residual = Y - L @ alpha
    residual_mat = residual@residual.T
    w1 = residual.T @ W1 @ residual/np.trace(W1@W1)#(residual.T @ W1 @ residual)**2/np.trace(W1@W1)/np.trace(residual_mat@H@residual_mat@H) 
    w0 = residual.T @ W0 @ residual/np.trace(W0@W0)#(residual.T @ W0 @ residual)**2/np.trace(W0@W0)/np.trace(residual_mat@H@residual_mat@H)
    # w0, w1 = 1/w0, 1/w1
    #w0 = np.trace(L@H@W0@H)
    #w1 = np.trace(L@H@W1@H)
    W = (w1 * W1 + w0 * W0)/(w1+w0)
    # LWL_inv = chol_inv(L @ W @ L + L / N2 + JITTER * EYEN)
    # alpha = LWL_inv @ L @ W @ Y
    
    reg = residual.T@H@L@H@residual# np.trace(residual_mat@H@L@H)
    
    return residual.T @ W @ residual + lambdaH * reg#alpha.T @ L @ alpha  # ((Y- L@alpha).T@W@(Y- L@alpha))**2/np.trace(W@W)/np.trace(residual_mat@H@residual_mat@H) + lambdaH*alpha.T@L@alpha

def CV_once(train_ad, dev_ad, al, lambdaH):
    EYEN = np.eye(train_ad.x.shape[0])
    H = EYEN - 1 / train_ad.x.shape[0]
    # ak = np.array([get_median_inter_mnist(train_ad.z[:, [i]]) for i in range(train_ad.z.shape[1])])
    # W = np.array([H @ (np.exp(-_sqdist(train_ad.z[:, [i]], None) / ak[i] ** 2 / 2) + 1e-6 * EYEN) @ H / train_ad.z.shape[0]**2 for i in
    #               range(train_ad.z.shape[1])])
    ak0 = get_median_inter_mnist(train_ad.z[:, [0]])
    ak1 = get_median_inter_mnist(train_ad.z[:, [1]])
    ntrain = train_ad.z.shape[0]
    W0 = H @ (np.exp(-_sqdist(train_ad.z[:, [0]], None) / ak0 ** 2 / 2) + 1e-6 * EYEN) @ H / ntrain ** 2
    W1 = H @ (np.exp(-_sqdist(train_ad.z[:, [1]], None) / ak1 ** 2 / 2) + 1e-6 * EYEN) @ H / ntrain ** 2

    assert len(al) == train_ad.x.shape[1]
    # L = np.exp(
    #     -np.sum([_sqdist(train_ad.x[:, [i]], None) / al[i] ** 2 for i in range(train_ad.x.shape[1])]) / 2) + 1e-6 * EYEN
    L = np.exp(-_sqdist(train_ad.x, None) / al ** 2 / 2)

    def MMR(alpha):
        alpha = alpha[:, None]
        residual = train_ad.y - L @ alpha
        #w0 = (residual.T @ W0 @ residual)**2 /np.trace(W0@W0)/np.trace(residual@residual.T@H@residual@residual.T@H)
        #w1 = (residual.T @ W1 @ residual)**2 /np.trace(W1@W1)/np.trace(residual@residual.T@H@residual@residual.T@H)
        # residual_mat = residual@residual.T
        w0 = residual.T @ W0 @ residual/np.trace(W0@W0)#/np.trace(residual_mat@H@residual_mat@H)# np.trace(L@W0)#
        w1 = residual.T @ W1 @ residual/np.trace(W1@W1)#/np.trace(residual_mat@H@residual_mat@H)# np.trace(L@W1)# 
        # w0, w1 = 1/w0, 1/w1
        w0,w1 = w0/(w0+w1),w1/(w0+w1)
        W = w0*W0+w1*W1
        reg =  residual.T@H@L@H@residual
        train_ad_loss = residual.T @ W @ residual+lambdaH * reg # alpha.T @ L @ alpha # lambdaH * alpha.T @ L @ alpha
        return train_ad_loss

    bounds = None
    params0 = np.random.randn(train_ad.x.shape[0]) / 10
    obj_grad = value_and_grad(lambda params: MMR(params))
    res = minimize(obj_grad, x0=params0, bounds=bounds, method='L-BFGS-B', jac=True, options={'maxiter': 500})

    alpha = res.x[:, None]
    EYEN = np.eye(dev_ad.x.shape[0])
    H = EYEN - 1 / dev_ad.x.shape[0]
    # dev_L = np.exp(-np.sum([_sqdist(dev_ad.x[:, [i]], train_ad.x[:, [i]]) / al[i] ** 2 for i in
    #                         range(dev_ad.x.shape[1])]) / 2) + 1e-6 * EYEN
    dev_L = np.exp(-_sqdist(dev_ad.x, train_ad.x) / al ** 2 / 2) + 1e-6 * EYEN
    ndev = dev_ad.z.shape[0]
    dev_W0 = H @ (np.exp(-_sqdist(dev_ad.z[:, [0]], None) / ak0 ** 2 / 2) + 1e-6 * EYEN) @ H / ndev ** 2
    dev_W1 = H @ (np.exp(-_sqdist(dev_ad.z[:, [1]], None) / ak1 ** 2 / 2) + 1e-6 * EYEN) @ H / ndev ** 2
    residual = dev_ad.y - dev_L @ alpha
    residual2 = train_ad.y - L @ alpha
    # w0 = np.trace(L@dev_W0)**2/np.trace(L@H@L@H)/np.trace(dev_W0@dev_W0)
    # w1 = np.trace(L@dev_W1)**2/np.trace(L@H@L@H)/np.trace(dev_W1@dev_W1)
    # residual_mat = residual2@residual2.T
    w0 = residual2.T @ W0 @ residual2/np.trace(W0@W0)#/np.trace(residual_mat@H@residual_mat@H)
    w1 = residual2.T @ W1 @ residual2/np.trace(W1@W1)#/np.trace(residual_mat@H@residual_mat@H)
    # w0, w1 = 1/w0, 1/w1
    w0,w1 = w0 / (w0 + w1), w1 / (w0 + w1)
    print('train weights:',w0,w1)
    # w0 = (residual.T @ dev_W0 @ residual)**2/np.trace(dev_W0@dev_W0)/np.trace(residual@residual.T@H@residual@residual.T@H)
    # w1 = (residual.T @ dev_W1 @ residual)**2/np.trace(dev_W1@dev_W1)/np.trace(residual@residual.T@H@residual@residual.T@H)
    # residual_mat = residual@residual.T
    w0 = residual.T @ dev_W0 @ residual/np.trace(dev_W0@dev_W0)#/np.trace(residual_mat@H@residual_mat@H)
    w1 = residual.T @ dev_W1 @ residual/np.trace(dev_W1@dev_W1)#/np.trace(residual_mat@H@residual_mat@H)
    # w0, w1 = 1/w0, 1/w1
    w0,w1 = w0 / (w0 + w1), w1 / (w0 + w1)
    print('dev weights:',w0,w1)
    # w = np.array(
    #     [(dev_ad.y - dev_L @ alpha).T @ dev_W[i] @ (dev_ad.y - dev_L @ alpha) for i in range(train_ad.z.shape[1])])
    # w /= np.sum(w, axis=0)

    # w = np.array([(train_ad.y - L @ alpha).T @ W[i] @ (train_ad.y - L @ alpha) for i in range(train_ad.z.shape[1])])
    # w = w / np.sum(w, axis=0)
    W = w0 * dev_W0 + w1 * dev_W1
    dev_loss = residual.T @ W @ residual
    return dev_loss

def callback0(alpha):
    global Nfeval
    if Nfeval % 100 == 0:
        alpha = alpha[:, None]
        pred_mean = test_L @ alpha
        test_err = ((pred_mean - test_G) ** 2).mean()
        # print('test err, norm: ', test_err, alpha.T@L@alpha)
    Nfeval += 1

    
N2 = X.shape[0] ** 2
W0 = _sqdist(Z[:, [0]], None)
W1 = _sqdist(Z[:, [1]], None)
# del W0
L0, test_L0 = _sqdist(X, None), _sqdist(test_X, X)
W0 = (np.exp(-W0 / ak0 ** 2 / 2) + 1e-6 * np.eye(W0.shape[0])) / N2
W1 = (np.exp(-W1 / ak1 ** 2 / 2) + 1e-6 * np.eye(W1.shape[0])) / N2
H = np.eye(W0.shape[0])-1/W0.shape[0]
W0, W1 = H@W0@H, H@W1@H

dev_loss_list = []
test_err_list = []
for al in [al0 * j for j in np.logspace(-1, 1, 6)]:
    al = np.array([al])
    for lambdaH in np.logspace(-1,1,6):
        global Nfeval
        Nfeval = 1
        print(' -------  lambda {} al {}  --------- '.format(lambdaH, al))
        dev_loss1 = CV_once(train_ad, dev_ad, al, lambdaH)
        dev_loss2 = CV_once(dev_ad, train_ad, al, lambdaH)
        # print('dev_loss:', (dev_loss1+dev_loss2)[0,0])
        dev_loss = dev_loss1+dev_loss2
        dev_loss_list += [dev_loss[0,0]]

        L = np.exp(-L0 / al ** 2 / 2) + 1e-6 * np.eye(L0.shape[0])
        test_L = np.exp(-test_L0 / al ** 2 / 2)
        alpha0 = np.random.randn(X.shape[0])
        bounds = None
        obj_grad = value_and_grad(lambda params: MMR(params, lambdaH))
        res = minimize(obj_grad, x0=alpha0, bounds=bounds, method='L-BFGS-B', jac=True, options={'maxiter': 1000})
        alpha = res.x[:, None]
        pred_mean = test_L @ alpha
        test_err = ((pred_mean - test_G) ** 2).mean()
        test_err_list += [test_err]
        print(dev_loss,test_err)



 -------  lambda 0.1 al [0.19381355]  --------- 
train weights: [[0.8337451]] [[0.1662549]]
dev weights: [[0.93280074]] [[0.06719926]]
train weights: [[0.98594762]] [[0.01405238]]
dev weights: [[0.8739812]] [[0.1260188]]
[[0.02970478]] 2.1213966114758467
 -------  lambda 0.251188643150958 al [0.19381355]  --------- 
train weights: [[0.83501505]] [[0.16498495]]
dev weights: [[0.93474414]] [[0.06525586]]
train weights: [[0.98545991]] [[0.01454009]]
dev weights: [[0.87471903]] [[0.12528097]]
[[0.030241]] 0.6060720052616962
 -------  lambda 0.6309573444801934 al [0.19381355]  --------- 
train weights: [[0.83458857]] [[0.16541143]]
dev weights: [[0.93481044]] [[0.06518956]]
train weights: [[0.98494372]] [[0.01505628]]
dev weights: [[0.87890112]] [[0.12109888]]
[[0.03075272]] 0.9433318799892254
 -------  lambda 1.584893192461114 al [0.19381355]  --------- 
train weights: [[0.83531935]] [[0.16468065]]
dev weights: [[0.93426599]] [[0.06573401]]
train weights: [[0.98591276]] [[0.01408724]]
dev 

KeyboardInterrupt: 

In [ ]:
plt.scatter(dev_loss_list,test_err_list)
plt.ylabel('test err')
plt.xlabel('dev loss')
plt.xlim(np.min(dev_loss_list)*0.95,np.max(dev_loss_list)*1.05)
plt.ylim(np.min(test_err_list)*0.95,1)
plt.show()

In [ ]:
np.min(test_err_list)

In [ ]:
np.argsort(test_err_list)

In [ ]:
inds = np.array(dev_loss_list).argsort()
np.array(test_err_list)[inds]

In [ ]:
np.array(test_err_list).reshape((6,6))